In [121]:
import pandas as pd
df=pd.read_csv('Churn_Modelling.csv')
df.drop(['RowNumber','Surname','CustomerId'],axis=1,inplace=True)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [122]:
import numpy as np
import tensorflow as tf
tf.__version__

'2.17.0'

# Data preprocessing

In [123]:
df["Gender"].isna().sum()

0

In [124]:
print(df["Gender"].unique())
def GenderEncoding(gender):
  if gender=="Female":
    return int(0)
  elif gender=="Male":
    return int(1)
Gender=df["Gender"].apply(GenderEncoding)
Gender

['Female' 'Male']


,Gender
0,0
1,0
2,0
3,0
4,0
...,...
9995,1
9996,1
9997,0
9998,1


In [125]:
Geography=pd.get_dummies(df["Geography"],dtype=int)
Geography

,France,Germany,Spain
0,1,0,0
1,0,0,1
2,1,0,0
3,1,0,0
4,0,0,1
...,...,...,...
9995,1,0,0
9996,1,0,0
9997,1,0,0
9998,0,1,0


In [126]:
df["Gender"]=Gender
df=pd.concat([df,Geography],axis='columns')
df.drop('Geography',axis=1,inplace=True)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Germany,Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0,0,1


In [137]:
from sklearn.preprocessing import StandardScaler
Scaler=StandardScaler()
df[["CreditScore","Tenure","NumOfProducts","Balance","EstimatedSalary"]]=Scaler.fit_transform(df[["CreditScore","Tenure","NumOfProducts","Balance","EstimatedSalary"]])
# df["CreditScore"]=Scaler.fit_transform(df["CreditScore"].values.reshape(-1,1))
# df["Tenure"]=Scaler.fit_transform(df["Tenure"].values.reshape(-1,1))
# df["NumOfProducts"]=Scaler.fit_transform(df["NumOfProducts"].values.reshape(-1,1))
# df["Balance"]=Scaler.fit_transform(df["Balance"].values.reshape(-1,1))
# df["EstimatedSalary"]=Scaler.fit_transform(df["EstimatedSalary"].values.reshape(-1,1))
df.head()


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Germany,Spain
0,-0.326221,0,4,-1.041760,-1.225848,-0.911583,1,1,0.021886,1,1,0,0
1,-0.440036,0,4,-1.387538,0.117350,-0.911583,0,1,0.216534,0,0,0,1
2,-1.536794,0,4,1.032908,1.333053,2.527057,1,0,0.240687,1,1,0,0
3,0.501521,0,3,-1.387538,-1.225848,0.807737,0,0,-0.108918,0,1,0,0
4,2.063884,0,4,-1.041760,0.785728,-0.911583,1,1,-0.365276,0,0,0,1


In [128]:
def AgeBins(age):
  return int(age/10)
df["Age"]=df["Age"].apply(AgeBins)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Germany,Spain
0,-0.326221,0,4,-1.041760,-1.225848,-0.911583,1,1,0.021886,1,1,0,0
1,-0.440036,0,4,-1.387538,0.117350,-0.911583,0,1,0.216534,0,0,0,1
2,-1.536794,0,4,1.032908,1.333053,2.527057,1,0,0.240687,1,1,0,0
3,0.501521,0,3,-1.387538,-1.225848,0.807737,0,0,-0.108918,0,1,0,0
4,2.063884,0,4,-1.041760,0.785728,-0.911583,1,1,-0.365276,0,0,0,1


In [129]:
from sklearn.model_selection import train_test_split
X=df.drop("Exited",axis=1)
y=df["Exited"]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [130]:
X_train.shape

(8000, 12)

# ANN model building

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

ann = Sequential()
ann.add(Dense(units=6, activation='relu', input_dim=12))
ann.add(Dense(units=6, activation='relu'))
ann.add(Dense(units=1, activation='sigmoid'))
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
ann.fit(X_train, y_train, batch_size=32, epochs=100)

In [138]:
X_test.columns

Index(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'France', 'Germany',
       'Spain'],
      dtype='object')

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: $ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: $ 50000

In [146]:
Geography = "France"
CreditScore = 600
Gender = "Male"
Age = 40
Tenure = 3
Balance = 60000
NumOfProducts = 2
HasCrCard = 1
IsActiveMember = 1
EstimatedSalary = 50000
Gender = GenderEncoding(Gender)
Age = AgeBins(Age)
test_input = pd.DataFrame([[CreditScore, Gender, Age, Tenure, Balance, NumOfProducts, HasCrCard, IsActiveMember, EstimatedSalary, Geography]],
                          columns=['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Geography'])

In [147]:
Geography_encoded = pd.get_dummies(test_input['Geography'],dtype=int)
test_input = pd.concat([test_input, Geography_encoded], axis=1)
test_input.drop('Geography', axis=1, inplace=True)
test_input

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,France
0,600,1,4,3,60000,2,1,1,50000,1


In [148]:
test_input = test_input.reindex(columns=X_train.columns, fill_value=0)
test_input

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,France,Germany,Spain
0,600,1,4,3,60000,2,1,1,50000,1,0,0


In [150]:
test_input[["CreditScore","Tenure","NumOfProducts","Balance","EstimatedSalary"]] = Scaler.transform(test_input[["CreditScore","Tenure","NumOfProducts","Balance","EstimatedSalary"]])
test_input

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,France,Germany,Spain
0,600.0,1,4,3.0,60000.0,2.0,1,1,50000.0,1,0,0


In [151]:
result = ann.predict(test_input)
result = (result > 0.5)
result

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


array([[False]])

# Predict the X_test

In [152]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
y_pred

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array([[False],
       [False],
       [False],
       ...,
       [ True],
       [False],
       [False]])

In [155]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
print("Accuracy: ", accuracy_score(y_test, y_pred)*100,"%")

[[1521   69]
 [ 225  185]]
Accuracy:  85.3 %
